<table>
    <tr>
        <td  style="background-color:#ffffff;" width="18%"><img src="..\images\qworld.jpg"  align="left"></td>
        <td style="background-color:#ffffff;" width="*"></td>
        <td  style="background-color:#ffffff;vertical-align:text-top;" width="21%"><img src="..\images\qpoland.jpg" align="right"></td>  
        <td  style="background-color:#ffffff;vertical-align:text-top;" width="19%" ><img src="..\images\qlatvia.jpg" align="right"></td>  
    </tr>
    <tr><td colspan="4" align="right" style="color:#777777;background-color:#ffffff;font-size:12px;">
        prepared by Anastasija Trizna
    </td></tr>
    <tr><td colspan="4" align="right" style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;">
        This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros.
    </td></tr>
</table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $

<h3>Sifting</h3>

We can now continue with classical post-processing.

Asja and Balvis should determine, by public exchange of messages, in which rounds their encoding and decoding bases agreed. 

During this stage, Balvis reveals the number and rounds when he was able receive a qubit from Asja and Asja share the bases she used to encode each qubit. 

Both users discard rounds when Balvis didn’t receive a qubit and when he choose measurement basis different from the one Asja used when prepared states. 

<h3>Task 1: Classical post-processing: Sifting</h3>

Continue your code from previous notebook and implement sifting phase:<ul>
    <li>Asja and Balvis compare bases they used and keep bits from rounds where bases matched. 
    <li>Both Asja and Balvis remove from their key string bits where they used different bases.
    <li>Print out the keys Asja and Balvis now have. 
</ul>

In [1]:
#
# move here your solution from task: Distributing quantum states
#

In [2]:
#
# your code for sifting is here
#

In [1]:
#Solution
# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

#Source: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb
def SendState(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    # Parse the instructions and apply to new circuit
    for instruction in qs:
        if instruction[0] == 'x':
            old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')

qreg = QuantumRegister(32) # quantum register with 8 qubits
creg = ClassicalRegister(32) # classical register with 8 bits

# Quantum circuit for Asja state
asja = QuantumCircuit(qreg, creg, name='Asja')

send=[] #Initial bit string ot send
asja_basis=[] #Register to save information about encoding basis
balvis_basis=[] #Register to save information about decoding basis

#Creating random bit string
for i in range(32):
    bit = randrange(2)
    send.append(bit)
    
#Preparing qubits, apply X gate if bit is equal 1
for i, n in enumerate(send):
    if n==1:
        asja.x(qreg[i]) # apply x-gate

#send_str = ''.join(str(e) for e in send)

#Encoding
for i in range(32):
    r=randrange(2) #Asja randomly pick a basis
    if r==0: #if bit is 0, then she encodes in Z basis
        asja_basis.append('Z')
    else: #if bit is 1, then she encodes in X basis
        asja.h(qreg[i])
        asja_basis.append('X')

balvis = QuantumCircuit(qreg, creg, name='Balvis') #Defining Balvis circuit
SendState(asja, balvis, 'Asja') #Asja sends states to Balvis

#Balvis measures qubits
for i in range(32):
    r=randrange(2) #Balvis randomly pick a basis
    if r==0: #if bit is 0, then measures in Z basis
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('Z')
    else: #if bit is 1, then measures in X basis
        balvis.h(qreg[i])
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('X')

job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
counts = job.result().get_counts(balvis) # counts is a dictionary object in python
counts = print_outcomes_in_reserve(counts)

#Saving Balvis received string as a list
received = list(map(int, counts))

print("Asja sent:", send)
print("Asja encoding basis:", asja_basis)
print("Balvis received:", received)
print("Balvis decoding basis:", balvis_basis)
#Classical post-processing: Sifting
asja_key=[] #Asjas register for matching rounds
balvis_key=[] #Balvis register for matching rounds
for j in range(0,len(asja_basis)): #Going through list of bases 
    if asja_basis[j] == balvis_basis[j]: #Comparing
        asja_key.append(send[j])
        balvis_key.append(received[j]) #Keeping key bit if bases matched
    else:
        pass #Discard round if bases mismatched

print("Asjas key =", asja_key)
print("Balvis key =", balvis_key)

Asja sent: [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1]
Asja encoding basis: ['Z', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z']
Balvis received: [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1]
Balvis decoding basis: ['Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X']
Asjas key = [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
Balvis key = [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]


<h3>Quantum Bit Error Correction</h3>

At this stage Asja randomly chooses $n$ of remain bits to test and says to Balvis which rounds are tested. Asja and Balvis exchange bits, compare and compute Quantum Bit Error Rate (QBER). Asja and Balvis estimate the information gained by eavesdropper during the quantum transmission stage from the observed QBER. 

<b>For noise-free BB84 version</b>

If QBER value is not zero - users abort the protocol. 

If threshold is not reached, they will continue with privacy amplification step.

<h3>Task 3: Classical post-processing - Quantum Bit Error Rate</h3>

Implement QBER phase:<ul>
    <li>Asja randomly choses testing round.
    <li>Asja and Balvis compare bit-by-bit $\frac{1}{3}$ of their final key (divide without remainder)
    <li>If their bits does not match - this round is considered as an error.
    <li>Calculate error rate as: $QBER=\frac{error}{compared bits}$
    <li>Discard bits used for QBER estimation from their final key strings.
    <li>Print keys Asja and Balvis now have and QBER value.
</ul>

Python hints:

<li>To divide without remained you can use //. Example: print(5//2) 
<li>Use <i>del</i> to specify the index of the element you want to delete. Example: del list[index]

In [4]:
#
# your code for QBER is here
#

In [2]:
#Solution
#QBER
rounds = len(asja_key)//3
errors=0
for i in range(rounds):
    bit_index = randrange(len(asja_key)) 
    tested_bit = asja_key[bit_index]
    print ("Asja randomly selected bit index - ", bit_index, "- and its value is - ", tested_bit)
    if asja_key[bit_index]!=balvis_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    del asja_key[bit_index] #removing tested bits from key strings
    del balvis_key[bit_index]
QBER=errors/rounds #calculating QBER
        
print("QBER value =", QBER)
print("Asjas secret key =", asja_key)
print("Balvis secret key =", balvis_key)

Asja randomly selected bit index -  0 - and its value is -  1
Asja randomly selected bit index -  0 - and its value is -  0
Asja randomly selected bit index -  2 - and its value is -  0
Asja randomly selected bit index -  9 - and its value is -  0
Asja randomly selected bit index -  5 - and its value is -  0
QBER value = 0.0
Asjas secret key = [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1]
Balvis secret key = [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1]
